<a href="https://colab.research.google.com/github/RefShah/paylah_3_23/blob/main/paylah_Fri_v1_03_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### To overcome missing 'Enter' key on some mobile devices, tick the 'Desktop site' option on your mobile browser

#### Run install cells only if your env is missing it
#### Else just goto import cell and select Runtime | Run after
#### After entering your local postal code, scroll all the way down to find your map

In [1]:
!pip install openlocationcode

  Preparing metadata (setup.py) ... done
  Created wheel for openlocationcode: filename=openlocationcode-1.0.1-py3-none-any.whl size=8401 sha256=1fb38ae1b206bf8b7ea4f0701a9f6eee3d35b5c395bdf09397af2cd8f3d76326
  Stored in directory: /root/.cache/pip/wheels/1a/35/cb/7683b12844d939a5bf6dc4a5b741a705970594a2160f5ed25f
Successfully built openlocationcode


In [2]:
!pip install pluscodepy

In [3]:
!pip install folium --upgrade

In [4]:
!pip install geocoder

In [5]:
from numpy import radians, cos, sin, sqrt
from numpy import arcsin as asin
import pandas as pd
import numpy as np
import re
import os
import glob
from datetime import datetime

import requests
import folium
from folium import plugins
import ipywidgets
import geocoder
from folium.features import DivIcon
import io
# import ipywidgets as widgets
import pluscodepy

In [6]:
# file_id = '1k86cN30yMQb5UNngylVqAEaMX8yjjHD7'
file_id = '1yxkB_CxR9nFMxsAzTzl1koi2q3W7XAh4'

url = f'https://drive.google.com/uc?id={file_id}'

# Download the contents of the CSV file
download = requests.get(url).content

# df_paylah = pd.read_csv(io.StringIO(download.decode('utf-8')))
df_paylah_latlong = pd.read_csv(io.StringIO(download.decode('utf-8')))
df_paylah_latlong.drop('Unnamed: 0', axis=1, inplace=True)

In [7]:
df_paylah_latlong.head()

,Column1,Column2.1,Postal Code,LATLONG
0,101 YISHUN CS,101 YISHUN AVE 5,760101,"('1.43053231016112', '103.827618665577')"
1,106 FOOD ALLIANCE,106 HOUGANG AVE 1,530106,"('1.35370313485788', '103.890680820842')"
2,112 BUKIT MERAH MARKET & FOOD CENTRE,112 JALAN BUKIT MERAH,160112,"('1.28000480135122', '103.82601329621')"
3,115 BUKIT MERAH VIEW MARKET AND HAWKER CENTRE,115 BUKIT MERAH VIEW,151115,"('1.28550445567433', '103.821823256087')"
4,115 FOOD VILLAGE,115 CANBERRA WALK,750115,"('1.44608188224858', '103.828739863564')"


In [8]:
converter = pluscodepy.Converter()

In [9]:
# type(response_me.json()['results'])
found = 0
while found == 0:
    my_loc_pc = input('Enter your SG postal code eg. 188720 >>> ')
    url_me = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={my_loc_pc}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response_me = requests.request("GET", url_me)
    print(response_me.text)
    found = len(response_me.json()['results'])
    if found == 0:
      # latitude, longitude = converter.decode(my_loc_pc + ' Singapore')\
      if not converter.decode(my_loc_pc + ' Singapore'):
        # print('F')
        found = 0
      else:
        latitude, longitude = converter.decode(my_loc_pc)
        print(f'Latitude: {latitude}, Longitude: {longitude}')
        found = 99

Enter your SG postal code eg. 188720 >>> 7QWR+8Q Singapore
{
  "found": 0,
  "totalNumPages": 0,
  "pageNum": 1,
  "results": []
}
getting for Singapore Singapore
getting for Singapore
Latitude: 1.2958125, Longitude: 103.7919375


In [10]:
if found != 99:
  lat_me = response_me.json()['results'][0]["LATITUDE"]
  lng_me = response_me.json()['results'][0]["LONGITUDE"]
else:
  lat_me = latitude
  lng_me = longitude

In [11]:
map_zoo = folium.Map()
map_zoo = folium.Map(location=[float(lat_me), float(lng_me)], zoom_start=17)
map_zoo

In [12]:
def get_LATLONG(astr):
    url_me = f"https://www.onemap.gov.sg/api/common/elastic/search?searchVal={astr}&returnGeom=Y&getAddrDetails=Y&pageNum=1"
    response_me = requests.request("GET", url_me)
    if response_me.json()['found'] < 1:
        print(astr)
    return response_me.json()['results'][0]["LATITUDE"], response_me.json()['results'][0]["LONGITUDE"]

In [13]:
# df_paylah["LATLONG"] = df_paylah["LATLONG"].map(str)
df_paylah_latlong[['LAT', 'LONG']] = df_paylah_latlong["LATLONG"].str.split(pat = ', ', expand = True, n=1)
df_paylah_latlong['LAT'] = df_paylah_latlong['LAT'].str.replace(r'^..', '',regex=True).str.replace(r'\'$', '',regex=True)
df_paylah_latlong['LONG'] = df_paylah_latlong['LONG'].str.replace(r'^.', '',regex=True).str.replace(r'\'.$', '',regex=True)
df_paylah_latlong.head()

,Column1,Column2.1,Postal Code,LATLONG,LAT,LONG
0,101 YISHUN CS,101 YISHUN AVE 5,760101,"('1.43053231016112', '103.827618665577')",1.43053231016112,103.827618665577
1,106 FOOD ALLIANCE,106 HOUGANG AVE 1,530106,"('1.35370313485788', '103.890680820842')",1.35370313485788,103.890680820842
2,112 BUKIT MERAH MARKET & FOOD CENTRE,112 JALAN BUKIT MERAH,160112,"('1.28000480135122', '103.82601329621')",1.28000480135122,103.82601329621
3,115 BUKIT MERAH VIEW MARKET AND HAWKER CENTRE,115 BUKIT MERAH VIEW,151115,"('1.28550445567433', '103.821823256087')",1.28550445567433,103.821823256087
4,115 FOOD VILLAGE,115 CANBERRA WALK,750115,"('1.44608188224858', '103.828739863564')",1.44608188224858,103.828739863564


In [14]:
def haversine_astr(astr):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    lon2=float(lng_me)
    lat2=float(lat_me)
    # convert decimal degrees to radians
#     lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    lat1, lon1, lat2, lon2 = map(radians, [astr['LONG'], astr['LAT'], lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [15]:
df_paylah_latlong['LAT'] = df_paylah_latlong['LAT'].map(float)
df_paylah_latlong['LONG'] = df_paylah_latlong['LONG'].map(float)
df_paylah_latlong['haversine'] = df_paylah_latlong.apply(haversine_astr, axis=1)
df_paylah_latlong.head()

,Column1,Column2.1,Postal Code,LATLONG,LAT,LONG,haversine
0,101 YISHUN CS,101 YISHUN AVE 5,760101,"('1.43053231016112', '103.827618665577')",1.430532,103.827619,5.341118
1,106 FOOD ALLIANCE,106 HOUGANG AVE 1,530106,"('1.35370313485788', '103.890680820842')",1.353703,103.890681,11.087225
2,112 BUKIT MERAH MARKET & FOOD CENTRE,112 JALAN BUKIT MERAH,160112,"('1.28000480135122', '103.82601329621')",1.280005,103.826013,3.812212
3,115 BUKIT MERAH VIEW MARKET AND HAWKER CENTRE,115 BUKIT MERAH VIEW,151115,"('1.28550445567433', '103.821823256087')",1.285504,103.821823,3.334384
4,115 FOOD VILLAGE,115 CANBERRA WALK,750115,"('1.44608188224858', '103.828739863564')",1.446082,103.828740,5.714501


In [16]:
dist_ = 0.35
# df_local = df_paylah.loc[df_paylah.haversine<dist_].copy()
df_local = df_paylah_latlong.loc[df_paylah_latlong.haversine<dist_].copy()
len(df_local)

4

In [17]:
df_local

,Column1,Column2.1,Postal Code,LATLONG,LAT,LONG,haversine
234,CHANG CHENG (40),40 HOLLAND DR,270040,"('1.30717590843196', '103.793500328714')",1.307176,103.793500,0.347774
454,HOLLAND DRIVE FOOD CENTRE,44 HOLLAND DRIVE,270044,"('1.30811780015992', '103.792773769341')",1.308118,103.792774,0.339200
649,MEALBOX,30 BIOPOLIS ST,138671,"('1.30323831850192', '103.79256511685')",1.303238,103.792565,0.208856
720,S11 (43),43 HOLLAND DR,270043,"('1.30737701594163', '103.792941409382')",1.307377,103.792941,0.326260


In [18]:
folium.Marker(location=[float(lat_me), float(lng_me)], popup='... are here', tooltip='You').add_to(map_zoo)
# for (index, row) in df_paylah.iterrows():
for (index, row) in df_local.iterrows():
  folium.Marker(location=[row.loc['LAT'], row.loc['LONG']], popup=row.loc['Column1'],
      icon=plugins.BeautifyIcon(#number=row.loc['Column1'],#i.icon_num,
                        icon='cutlery',
                        border_color='blue',
                        border_width=1,
                        text_color='red'#,
#                         inner_icon_style='margin-top:3px;')#).add_to(map_enum_icons)
                        )#).add_to(map_enum_icons)

        ).add_to(map_zoo)
  map_zoo.add_child(folium.CircleMarker(location=[row.loc['LAT'], row.loc['LONG']], radius=15))

# display map
map_zoo

In [19]:
# map_zoo.save('myloc.html')